# Libraries

In [1]:
%pip install -q -U datasets transformers accelerate sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 85.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import os
import random
import numpy as np
import torch
import transformers
from pprint import pprint
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments
from sklearn.metrics import f1_score

# Config

In [3]:
seed = 42
lang = 'sun'

hf_model_id = 'alxxtexxr/RoBERTa-Base-SE2025T11A-sun-v20241227113119' # BEST
hf_tokenizer_id = hf_model_id
hf_data_id = 'alxxtexxr/SemEval2025-Task11-Dataset'
hf_data_config = 'track_a_sun_70_15_15_stratify_v2'

In [4]:
def set_seed(seed):
    # Set random seed for NumPy
    np.random.seed(seed)

    # Set random seed for Torch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU
    torch.backends.cudnn.deterministic = True  # Ensures deterministic results
    torch.backends.cudnn.benchmark = False  # Avoids non-deterministic algorithms

    # Set random seed for Transformers
    transformers.set_seed(seed)

    # Optionally set random seed for sklearn and Python's own random module
    random.seed(seed)

    # Set random seed for os
    os.environ['PYTHONHASHSEED'] = str(seed)

    print(f"Random seed set to: {seed}")

set_seed(seed)

Random seed set to: 42


# Data

## Load Data

In [5]:
datasets = load_dataset(hf_data_id, hf_data_config)

cols = list(datasets['train'].features)
emotion_cols = [col for col in cols if col not in ['Unnamed: 0', 'text', 'emotion']]
splits = [*datasets.keys()]

print("Data columns:", cols)
print("Emotions columns:", emotion_cols)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md:   0%|          | 0.00/2.78k [00:00<?, ?B/s]

(…)ack_a/sun_70_15_15_stratify_v2/train.csv:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

(…)track_a/sun_70_15_15_stratify_v2/val.csv:   0%|          | 0.00/13.3k [00:00<?, ?B/s]

(…)rack_a/sun_70_15_15_stratify_v2/test.csv:   0%|          | 0.00/12.9k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Data columns: ['text', 'emotion', 'marah', 'jijik', 'takut', 'senang', 'sedih', 'terkejut', 'biasa']
Emotions columns: ['marah', 'jijik', 'takut', 'senang', 'sedih', 'terkejut', 'biasa']


In [6]:
class2id = {class_:id for id, class_ in enumerate(emotion_cols)}
id2class = {id:class_ for class_, id in class2id.items()}

print("Class to ID:")
pprint(class2id, width=1)
print()
print("ID to Class:")
pprint(id2class, width=1)

Class to ID:
{'biasa': 6,
 'jijik': 1,
 'marah': 0,
 'sedih': 4,
 'senang': 3,
 'takut': 2,
 'terkejut': 5}

ID to Class:
{0: 'marah',
 1: 'jijik',
 2: 'takut',
 3: 'senang',
 4: 'sedih',
 5: 'terkejut',
 6: 'biasa'}


## Preprocess Data

In [7]:
tokenizer = AutoTokenizer.from_pretrained(hf_tokenizer_id)

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/786k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/445k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.53M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

In [8]:
def one_hot_encode_emotion(emotion, emotion_cols):
    emotions = emotion.replace(" ", "").split(",")
    one_hot_emotion = [1.0 if emotion_col in emotions else 0.0 for emotion_col in emotion_cols] # Ensure that the label is float, not int
    return one_hot_emotion

def preprocess_function(data):
   text = data['text']
   emotion = data['emotion']
   labels = one_hot_encode_emotion(emotion, emotion_cols)
   data = tokenizer(text, truncation=True)
   data['labels'] = labels
   return data

tokenized_datasets = {split: datasets[split].map(preprocess_function) for split in splits}

Map:   0%|          | 0/668 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

In [9]:
# Sanity check
data = tokenized_datasets['train'][5]

print("Text:", data['text'])
print("Emotion(s):", data['emotion'])
print("Labels:", data['labels'], '-->', emotion_cols)

Text: ☝️Cung urang Kuningan, Pas urang waduk Darma, Kpan atuh main ka waduk Darma Dai, ??? Di antos, Aa sareng tteh Na, 🤗🤗🤗
Emotion(s): senang
Labels: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0] --> ['marah', 'jijik', 'takut', 'senang', 'sedih', 'terkejut', 'biasa']


In [10]:
data_collator = DataCollatorWithPadding(tokenizer)

# Model

In [11]:
model = AutoModelForSequenceClassification.from_pretrained(
    hf_model_id, 
    num_labels=len(emotion_cols),
    id2label=id2class, label2id=class2id,
    problem_type = "multi_label_classification",
)
print(model.classifier)

config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

RobertaClassificationHead(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (out_proj): Linear(in_features=768, out_features=7, bias=True)
)


# Evaluation

In [12]:
# threshold = 0.975 # BEST: 0.2

def sigmoid(x):
   return 1/(1 + np.exp(-x))

def compute_metrics(eval_pred, threshold=0.5):
    predictions, labels = eval_pred
    probs = sigmoid(predictions)
    y_pred = (probs > threshold).astype(int)
    y_true = labels.astype(int)

    # Compute F1 score for each type of averaging method
    f1_micro = f1_score(y_true, y_pred, average='micro', zero_division=0.0)
    f1_macro = f1_score(y_true, y_pred, average='macro', zero_division=0.0)
    f1_weighted = f1_score(y_true, y_pred, average='weighted', zero_division=0.0)
    f1_samples = f1_score(y_true, y_pred, average='samples', zero_division=0.0)
    f1_labels = f1_score(y_true, y_pred, average=None, zero_division=0.0)
    f1_labels_dict = {f'f1_label_{emotion_cols[i]}': f1_labels[i] for i in range(len(f1_labels))}

    return {
        'f1_macro': f1_macro,
        'f1_micro': f1_micro,
        'f1_weighted': f1_weighted,
        'f1_samples': f1_samples,
        **f1_labels_dict,
    }

In [13]:
for i in range(1, 10):
    threshold = i/10

    train_args = TrainingArguments(
        # Training config
        per_device_train_batch_size=2,
        # num_train_epochs=3,
        num_train_epochs=1,
        learning_rate=2e-5,
        weight_decay=0.01,

        # Logging config for training
        logging_strategy='steps',
        logging_steps=100,

        # Evaluation config during training
        per_device_eval_batch_size=2,
        eval_strategy='steps',
        eval_steps=100,

        # Model saving config
        # output_dir=project_name,
        output_dir=hf_model_id,
        save_strategy='epoch',
        # load_best_model_at_end=True,
    )

    trainer = Trainer(
        model=model,
        args=train_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['val'],
        processing_class=tokenizer,
        data_collator=data_collator,
        compute_metrics=(lambda eval_pred: compute_metrics(eval_pred, threshold)),
    )

    eval = trainer.evaluate(eval_dataset=tokenized_datasets['test'])

    print("="*64)
    print(f"THRESHOLD: {threshold}")
    print("="*64)
    print("Evaluation to copy:")
    f1_keys = [eval_key for eval_key in eval.keys() if 'f1' in eval_key]
    for i, k in enumerate(f1_keys): print(eval[k], end="\t" if i + 1 < len(f1_keys) else "")


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: alimtegar. Use `wandb login --relogin` to force relogin


THRESHOLD: 0.1
Evaluation to copy:
0.5257702534647853	0.6799116997792495	0.7502560816452186	0.68203125	0.5333333333333333	0.36363636363636365	0.0	0.8962264150943396	0.8275862068965517	0.34532374100719426	0.7142857142857143

THRESHOLD: 0.2
Evaluation to copy:
0.5683345570759915	0.7880434782608695	0.7928481715221136	0.8169270833333333	0.5454545454545454	0.5	0.0	0.9170731707317074	0.8421052631578947	0.5070422535211268	0.6666666666666666

THRESHOLD: 0.3
Evaluation to copy:
0.540569917886991	0.7962962962962963	0.7687352188700827	0.83046875	0.4	0.5	0.0	0.91	0.8076923076923077	0.43902439024390244	0.7272727272727273

THRESHOLD: 0.4
Evaluation to copy:
0.5221174183959298	0.7884615384615384	0.753826393302419	0.8096354166666666	0.4444444444444444	0.5	0.0	0.8979591836734694	0.8235294117647058	0.3888888888888889	0.6

THRESHOLD: 0.5
Evaluation to copy:
0.4351465867077221	0.7651006711409396	0.709747342191583	0.7822916666666666	0.4444444444444444	0.2857142857142857	0.0	0.9119170984455959	0.7659574468085106	0.1935483870967742	0.4444444444444444

THRESHOLD: 0.6
Evaluation to copy:
0.3380931273218507	0.7448275862068966	0.6593971631205674	0.7513020833333333	0.25	0.0	0.0	0.90625	0.7659574468085106	0.0	0.4444444444444444

THRESHOLD: 0.7
Evaluation to copy:
0.3115849600709849	0.7386759581881533	0.6495341614906832	0.7382812499999999	0.25	0.0	0.0	0.90625	0.7391304347826086	0.0	0.2857142857142857

THRESHOLD: 0.8
Evaluation to copy:
0.2718361218361219	0.7266187050359713	0.6370031796502386	0.7018229166666665	0.25	0.0	0.0	0.8972972972972973	0.7555555555555555	0.0	0.0

THRESHOLD: 0.9
Evaluation to copy:
0.26109634551495015	0.6992481203007519	0.6180424076607387	0.6523437499999999	0.25	0.0	0.0	0.88	0.6976744186046512	0.0	0.0